In [1]:
import os
import copy
import time
from glob import glob
import PIL
from pathlib import Path
from string import ascii_uppercase as letters
from collections import deque
from matplotlib import pyplot as plt
from typing import List,Tuple
from enum import Enum
from webcolors import hex_to_rgb, CSS3_HEX_TO_NAMES

UP, RIGHT, DOWN, LEFT = 0,1,2,3

from collections.abc import MutableSet

class OrderedSet(MutableSet):

    def __init__(self, iterable=None):
        self.end = end = [] 
        end += [None, end, end]
        self.map = {}
        if iterable is not None:
            self |= iterable

    def __len__(self):
        return len(self.map)

    def __contains__(self, key):
        return key in self.map

    def add(self, key):
        if key not in self.map:
            end = self.end
            curr = end[1]
            curr[2] = end[1] = self.map[key] = [key, curr, end]

    def discard(self, key):
        if key in self.map:        
            key, prev, next = self.map.pop(key)
            prev[2] = next
            next[1] = prev

    def __iter__(self):
        end = self.end
        curr = end[2]
        while curr is not end:
            yield curr[0]
            curr = curr[2]

    def __reversed__(self):
        end = self.end
        curr = end[1]
        while curr is not end:
            yield curr[0]
            curr = curr[1]

    def pop(self, last=True):
        if not self:
            raise KeyError('set is empty')
        key = self.end[1][0] if last else self.end[2][0]
        self.discard(key)
        return key

    def __repr__(self):
        if not self:
            return '%s()' % (self.__class__.__name__,)
        return '%s(%r)' % (self.__class__.__name__, list(self))

    def __eq__(self, other):
        if isinstance(other, OrderedSet):
            return len(self) == len(other) and list(self) == list(other)
        return set(self) == set(other)

def get_pixels(image):
    pixels = []
    for y in range(image.height):
        row = []
        for x in range(image.width):
            pixel = image.getpixel((x,y))[:-1]
            row.append(pixel)
        pixels.append(row)
    return pixels

class Color:
    def __init__(self, name:str, rgb:Tuple[int,int,int], alias:str):
        self.name = name
        self.r, self.g, self.b = rgb
        self.alias = alias
    
    def asTuple(self):
        return (self.r, self.g, self.b)
    
    def asHex(self):
        return str('#%02x%02x%02x' % (self.r, self.g, self.b)).upper()

    def __repr__(self):
        return f'{self.alias}{self.asHex()}'

    def __hash__(self):
        return hash(self.asHex())

    def __eq__(self, other):
        return self.asHex() == other.asHex()

class Colors:
    def __init__(self):
        self.l = set()
    
    def add(self, c:Color):
        self.l.add(c)
        
    def get(self, query:str|Tuple[int,int,int]):
        if type(query) == str:
            if len(query) == 1:
                for e in self.l:
                    if e.alias == query:
                        return e
            else:
                for e in self.l:
                    if e.name == query:
                        return e
        else:
            for e in self.l:
                if e.asTuple() == query:
                    return e
            pass
        return None

    def __len__(self):
        return len(self.l)
    
    def __repr__(self):
        return str(self.l)
    
    def __iter__(self):
        for e in self.l:
            yield e

def closest_colour(requested_colour):
    min_colours = {}
    for key, name in CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = hex_to_rgb(key)
        rd = (r_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        bd = (b_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]

def get_colors(image) -> Colors:
    colors = Colors()
    for prow in get_pixels(image):
        for pixel in prow:
            index = len(colors)
            color = Color(closest_colour(pixel), pixel, letters[index])
            colors.add(color)
    return colors

tile_colors = Colors()
tile_colors # [Alias] # [Hex]

class Tile:
    def __init__(self, image:str):
        self.image = PIL.Image.open(image)
        self.pixels = get_pixels(self.image)
        self.sockets = deque([[],[],[],[]])
        self.uid = -1
    
    def display(self):
        plt.axis('off')
        plt.imshow(self.image)
    
    def calculate_sockets(self, ):
        self.sockets = deque([[],[],[],[]])
        for x in range(len(self.pixels)):
            up_color = self.pixels[0][x]
            up_color = tile_colors.get(up_color)
            self.sockets[UP].append(up_color)

            down_color = self.pixels[len(self.pixels)-1][len(self.pixels)-x-1]
            down_color = tile_colors.get(down_color)
            self.sockets[DOWN].append(down_color)

        for y in range(len(self.pixels)):
            left_color = self.pixels[len(self.pixels)-y-1][0]
            left_color = tile_colors.get(left_color)
            self.sockets[LEFT].append(left_color)

            right_color = self.pixels[y][len(self.pixels)-1]
            right_color = tile_colors.get(right_color)
            self.sockets[RIGHT].append(right_color)

        self.uid = self.generate_uid()

    def generate_uid(self):
        h = ''
        for row in self.pixels:
            for pixel in row:
                alias = tile_colors.get(pixel).alias
                h += alias
            h += '\n'
        self.uid = hash(h)
  
    def __hash__(self):
        self.generate_uid()
        return self.uid
    
    def __eq__(self, other):
        return hash(self) == hash(other)

    def rotateOneClockwise(self):
        self.image = self.image.rotate(-90)
        self.pixels = get_pixels(self.image)
        self.sockets.rotate(1)
        self.generate_uid()
        return self

    def rotate(self, n:int): # ALWAYS CLOCKWISE
        for i in range(n):
            self.rotateOneClockwise()
        return self
        
    def copy(self):
        return copy.copy(self)
   
    def __repr__(self):
        self.display()
        return ''


def get_all_colors(tiles:List[Tile]):
    output = Colors()
    for tile in tiles:
        pixels = tile.pixels
        for row in pixels:
            for pixel in row:
                index = len(output)
                color = Color(closest_colour(pixel), pixel, letters[index])
                output.add(color)
    return output




```python
tiles = []
for tile in os.listdir('./tiles/'):
    tiles.append(Tile(f'./tiles/{tile}'))

tile_colors = get_all_colors(tiles)

for tile in tiles:
    tile.calculate_sockets()

stiles = OrderedSet()
for tile in tiles:
    for i in range(4):
        stiles.add(tile.rotate(1).copy())
        
stiles = list(stiles)

Path('tiles-new').mkdir(parents=True, exist_ok=True)
for i, t in enumerate(stiles):
    image = t.image
    image.save(f'./tiles-new/tile_{i}.png')
```

In [2]:
import json

In [3]:
tiles = []
for tile in os.listdir('./tiles-new/'):
    filename = f'./tiles-new/{tile}'
    if os.path.isfile(filename):
        if filename[-3:] == 'png':
            tiles.append(Tile(filename))

In [4]:
tile_colors = get_all_colors(tiles)
tile_colors

{A#12DC37, C#D7C660, B#240C34, D#706D5C}

In [5]:
for tile in tiles:
    tile.calculate_sockets()

In [7]:
def pixel_matrix_to_string(pixels):
    for row in pixels:
        for pixel in row:
            pass

In [17]:
colordict = dict()
for color in tile_colors:
    alias = color.alias
    r,g,b, = color.asTuple()
    name = color.name
    colordict[alias] = {'rgb':[r,g,b], 'name':name}
colordict

{'A': {'rgb': [18, 220, 55], 'name': 'limegreen'},
 'C': {'rgb': [215, 198, 96], 'name': 'darkkhaki'},
 'B': {'rgb': [36, 12, 52], 'name': 'midnightblue'},
 'D': {'rgb': [112, 109, 92], 'name': 'dimgray'}}

In [18]:
json.dumps(colordict)

'{"A": {"rgb": [18, 220, 55], "name": "limegreen"}, "C": {"rgb": [215, 198, 96], "name": "darkkhaki"}, "B": {"rgb": [36, 12, 52], "name": "midnightblue"}, "D": {"rgb": [112, 109, 92], "name": "dimgray"}}'